In [1]:
import numpy as np
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
user_data = pd.read_csv('Data/user_data.csv')
ads_data = pd.read_csv('Data/ads_data.csv')
user_messages = pd.read_csv('Data/user_messages.csv')
user_messages_test = pd.read_csv('Data/user_messages_test.csv')
ads_data = ads_data.loc[ads_data.enabled==1]

In [3]:

# user_data.loc[user_data.event==1]['images_count']

# user_data.head(50)
# ads_data.head()
# user_messages.head()
# user_messages_test.head()
# pd.unique(user_data.origin)
# ads_data.shape
# len(pd.unique(ads_data.ad_id))
# user_data.loc[user_data.user_id==541]

# ads_data.head()
s1 = set(pd.unique(ads_data.ad_id))
s2 = set(pd.unique(user_data.ad_id))
len(s1.union(s2))
len(pd.unique(user_data.user_id))

645168

12365

In [4]:
from sklearn.preprocessing import LabelEncoder
le_user_id = LabelEncoder()
user_data['encoded_user_id'] = le_user_id.fit_transform(user_data.user_id)

In [5]:
le_ad_id = LabelEncoder()
le_ad_id.fit(list(set(user_data.ad_id).union(set(ads_data.ad_id))))
user_data['encoded_ad_id'] = le_ad_id.transform(user_data.ad_id)
ads_data['encoded_ad_id'] = le_ad_id.transform(ads_data.ad_id)

LabelEncoder()

In [6]:
le_cat_id = LabelEncoder()
ads_data['encoded_category_id'] = le_cat_id.fit_transform(ads_data.category_id)

## -------------------------

In [7]:
from scipy import sparse

In [8]:
nu = len(pd.unique(user_data.user_id))
s1 = set(pd.unique(ads_data.ad_id))
s2 = set(pd.unique(user_data.ad_id))
na = len(s1.union(s2))
n = user_data.shape[0]
user_ad_mat = sparse.coo_matrix( ( np.ones(n), (user_data.encoded_user_id, user_data.encoded_ad_id) ) ,shape=[nu,na])
user_ad_mat = user_ad_mat.tocsr()

In [9]:
nc = len(pd.unique(ads_data.category_id))
n = ads_data.shape[0]
ad_cat_mat = sparse.coo_matrix( ( np.ones(n), ( ads_data.encoded_ad_id, ads_data.encoded_category_id )  ), shape=[na,nc] )
ad_cat_mat = ad_cat_mat.tocsr()

## M1 = U-A-U-A
## M2 = U-A-U-A-U-A
## M3 = U-A-C-A
## M4 = U-A-C-A-U-A

In [16]:
from scipy.io import savemat
from scipy.io import loadmat

In [17]:
savemat('matrices', {'user_ad_mat': user_ad_mat, 'ad_cat_mat': ad_cat_mat})

In [15]:
M3

<10x10 sparse matrix of type '<type 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Column format>

In [ ]:
user_data.replace(to_replace={'event':{'view':0, 'first_message':1}}, inplace=True)
user_data.replace(to_replace={'origin':{'browse_search':0, 'search':0, 'browse':0, 'home':1,
                                       'notification_center':1, 'push':2, 'drawer':2, 'deeplink':1}}, inplace=True)


In [ ]:
user_data['event_time'] = pd.to_datetime(user_data.event_time)
user_data['event_month'] = user_data['event_time'].dt.month
user_data['event_day'] = user_data['event_time'].dt.day
user_data['hour'] = user_data['event_time'].dt.hour

In [ ]:
ad_id = ads_data['ad_id']
dat = ads_data['category_id']
augmented_ads_data = pd.DataFrame(data=np.array(dat), index=ad_id)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
cols_norm = ['ad_messages', 'ad_views', 'ad_impressions']
n_cols_norm = ['n_' + x for x in cols_norm]
user_data[n_cols_norm] = user_data[cols_norm].apply(lambda x: (x-x.min())/(x.max()-x.min()))

In [ ]:
ratio = [0.85, 0.2, 0.15]
i=0
temp = pd.Series(np.zeros([user_data.shape[0]]) )
for label in n_cols_norm:
    temp += ratio[i]*user_data[label]
    i+=1

user_data['composition']=temp

In [ ]:
# user_data = user_data.sort_values(by=['origin', 'event_month', 'event_day', 'ad_messages', 'ad_views', 'ad_impressions'], ascending=[True, False, False, False, False, False])
# user_data = user_data.sort_values(by=['origin','composition'], ascending=[True, False])
user_data = user_data.sort_values(by=['event_day', 'ad_messages', 'ad_views', 'ad_impressions', 'event'], ascending=[ False, False, False, False, False])


In [ ]:
a = augmented_ads_data.loc[np.array(user_data['ad_id'])] 

In [ ]:
ads_for_cat={}
for c in pd.unique(user_messages_test.category_id):
    ad_id_for_c = a.loc[a[0]==c]     #  refer above cell for a
    ads_for_cat[c] = list(ad_id_for_c.index.drop_duplicates()[:10])
    

In [ ]:
ads_id = pd.Series(np.array(np.arange(user_messages_test.shape[0])))
i=0
for u, c in user_messages_test.values:
    b = user_data.loc[user_data.user_id==u]
#     b = b.sort_values(by=['ad_messages', 'ad_views', 'ad_impressions'], ascending=[False, False, False])
    user_ads = pd.unique(b['ad_id'])
    
    category_ads = ads_data.loc[ads_data.category_id==c]
    cate_ads = pd.unique(category_ads['ad_id'])
    temp=[]
    j=0
    for ad in user_ads:
        if (ad in cate_ads and j<10):
            temp.append(ad)
            j+=1
    
    t0 = len(temp)
    if(t0<10):
        t1 = 10-t0
        t2 = ads_for_cat.get(c)[:t1]
        temp = temp + t2
#     print len(temp)
    ads_id[i] = str(temp)
    
    i+=1
    
    
    

In [ ]:
submission = user_messages_test.copy()
submission['ads'] = ads_id

In [ ]:
submission.to_csv('neww_subm16.csv', index=False)

## ----------------------------------------------------------------------------------------------------------------------------------

In [ ]:
user_messages = pd.read_csv('Data/user_messages.csv')
i=0
for (u,c, l in user_messages.values and i<5):
    user_data.loc[np.logical_and(user_data.user_id==u)]
    i+=1